# Connect MySQL Using Python 

In [1]:
import pandas as pd 
import numpy as np 
import datetime
import mysql.connector 
from sqlalchemy import create_engine

%run auth.py

## Use mysql.connector 

In [2]:
database = 'employees'

cnx = mysql.connector.connect(**config,database=database)
cursor = cnx.cursor()

query = ("SELECT first_name, last_name, hire_date FROM employees "
             "WHERE hire_date BETWEEN %s AND %s;")

hire_start = datetime.date(1999, 1, 1)
hire_end = datetime.date(1999, 1, 2)

cursor.execute(query, (hire_start, hire_end))

for (first_name, last_name, hire_date) in cursor:
    print("{}, {} was hired on {:%d %b %Y}".format(last_name, first_name, hire_date))

cursor.close()
cnx.close()

Nourani, Ewing was hired on 01 Jan 1999
Auria, Hongzue was hired on 01 Jan 1999
Covnot, Filipp was hired on 02 Jan 1999
Piveteau, Jianhua was hired on 02 Jan 1999
Melton, Yuqun was hired on 01 Jan 1999
Ramsey, Duke was hired on 02 Jan 1999
Journel, Edwin was hired on 02 Jan 1999
Sullins, Gal was hired on 02 Jan 1999
Hellwagner, Martina was hired on 01 Jan 1999
Nittel, Supot was hired on 02 Jan 1999
Benaini, Eben was hired on 01 Jan 1999
Jahnichen, Abdelaziz was hired on 02 Jan 1999
Picco, Younwoo was hired on 01 Jan 1999


In [3]:
def run_query(query):
    cnx = mysql.connector.connect(**config,database=database)
    cursor = cnx.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()
    df = pd.DataFrame(rows)
    cursor.close()
    cnx.close()
    return df 

In [4]:
run_query("SHOW TABLES;")

,0
0,current_dept_emp
1,departments
2,dept_emp
3,dept_emp_latest_date
4,dept_manager
5,employees
6,salaries
7,titles
8,v_full_departments
9,v_full_employees


In [5]:
run_query(
    "SELECT first_name, last_name "
    "FROM employees WHERE hire_date = (SELECT MAX(hire_date) FROM employees);"
)

,0,1
0,Bikash,Covnot


In [6]:
run_query("SELECT title, COUNT(*) FROM titles GROUP BY 1 ORDER BY 1;")

,0,1
0,Assistant Engineer,15128
1,Engineer,115003
2,Manager,24
3,Senior Engineer,97750
4,Senior Staff,92853
5,Staff,107391
6,Technique Leader,15159


## Use sqlalchemy.create_engine

In [7]:
database = 'employees'
engine = create_engine(uri+database, echo=False)

In [8]:
def sql_to_pd(query):
    df = pd.read_sql(query, engine)
    return df 

In [9]:
sql_to_pd('SELECT title, COUNT(*) AS number_hired FROM titles GROUP BY 1 ORDER BY 1;')

,title,number_hired
0,Assistant Engineer,15128
1,Engineer,115003
2,Manager,24
3,Senior Engineer,97750
4,Senior Staff,92853
5,Staff,107391
6,Technique Leader,15159


In [10]:
sql_to_pd('SELECT title, COUNT(*) AS number_hired FROM titles GROUP BY 1 ORDER BY 1;')

,title,number_hired
0,Assistant Engineer,15128
1,Engineer,115003
2,Manager,24
3,Senior Engineer,97750
4,Senior Staff,92853
5,Staff,107391
6,Technique Leader,15159


In [11]:
query1=("SELECT YEAR(from_date) AS year, "
        "SUM(CASE WHEN title = 'Assistant Engineer' THEN 1 ELSE 0 END) AS 'AssistantEngineer', "
        "SUM(CASE WHEN title = 'Engineer' THEN 1 ELSE 0 END) AS 'Engineer', "
        "SUM(CASE WHEN title = 'Manager' THEN 1 ELSE 0 END) AS 'Manager', "
        "SUM(CASE WHEN title = 'Senior Engineer' THEN 1 ELSE 0 END) AS 'SeniorEngineer', "
        "SUM(CASE WHEN title = 'Senior Staff' THEN 1 ELSE 0 END) AS 'SeniorStaff', "
        "SUM(CASE WHEN title = 'Staff' THEN 1 ELSE 0 END) AS 'Staff', "
        "SUM(CASE WHEN title = 'Technique Leader' THEN 1 ELSE 0 END) AS 'TechniqueLeader' " 
        "FROM titles GROUP BY 1 ORDER BY 1; ") 

In [12]:
sql_to_pd(query1)

,year,AssistantEngineer,Engineer,Manager,SeniorEngineer,SeniorStaff,Staff,TechniqueLeader
0,1985,948.0,6472.0,9.0,1835.0,1665.0,6467.0,897.0
1,1986,989.0,6914.0,0.0,2017.0,1733.0,7169.0,1056.0
2,1987,980.0,7125.0,0.0,2051.0,1738.0,7136.0,957.0
3,1988,976.0,7023.0,2.0,2073.0,1825.0,7195.0,1013.0
4,1989,1047.0,7058.0,2.0,2050.0,1758.0,7363.0,1036.0
5,1990,1019.0,7267.0,0.0,3224.0,2982.0,7172.0,1005.0
6,1991,981.0,7261.0,4.0,4555.0,4239.0,7193.0,1041.0
7,1992,1047.0,7672.0,4.0,5828.0,5475.0,7202.0,1058.0
8,1993,1033.0,7671.0,0.0,6972.0,6915.0,7176.0,1017.0
9,1994,1010.0,7855.0,1.0,8141.0,8153.0,7243.0,964.0
